In [97]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer  
import re
from nltk import word_tokenize, sent_tokenize
import math 

In [98]:
import random
import tensorflow_hub as hub
import scipy.spatial
from nltk.translate.bleu_score import sentence_bleu

In [99]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

### Importing data 

In [100]:
#df = pd.read_csv("/content/drive/MyDrive/Reviews.csv")
df = pd.read_csv('Reviews.csv')




In [101]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [102]:
df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [103]:
df.shape

(568454, 10)

In [104]:
df = df.dropna()

### Data Pre-processing

In [105]:
df.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [106]:
df = df.drop(df.columns[:-2],axis=1)
df.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


In [107]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [108]:
stop_words = set(stopwords.words("english"))

In [109]:
# http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [110]:
def cleanText(text):
  text = text.lower()
  
  text = text.split('.')
  tt=[]
  stops = set(stopwords.words("english"))
  #text = [w for w in text if not w in stops]
  # text = " ".join(text)
  #for i in text:
  #  if i in contractions:
  #    arr.append(contractions[i])
  #  else:
  #    arr.append(i)
    
  for s in text:
    ss = []
    for w in s.split():
        if w not in stops and w not in contractions:
            ss.append(w)
        elif w not in stops and w in contractions:
            ss.append(contractions[w])
    sss = ' '.join(ss)
    tt.append(sss)
  text = ". ".join(tt)
  
  #text = re.sub(r'https?:\/\/*[\r\n]*', '', text, flags=re.MULTILINE)
  #text = re.sub(r'<br />', ' ', text)
  #text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
  #text = re.sub(r'\'', ' ', text)
  #text = re.sub(r'\<a href', ' ', text)
  #text = re.sub(r'&amp;', '', text)
  #text = re.sub('\s+',' ',text)
  return text

In [111]:
# Decreasing size of dataset

df_s = pd.DataFrame(random.sample(df.values.tolist(), 1000), columns = ["Summary", "Text"])

In [112]:
df_s 

,Summary,Text
0,"My wife loves these, so I love them too.",I have ensured my wife's love for me by buying...
1,EXCELLENT ANTI -OXIDANT,tHIS POWERFUL FRUIT HAS 2 TIMES THE ANTI-OXIDA...
2,tone's chili powder 20oz shaker,Never had a problem with this brand. i use alo...
3,"Better than Peanut M&M""s",I picked these up recently and the are amazing...
4,What's in this stuff?,What's in this stuff? Where's the list of ing...
...,...,...
995,These are the BEST PRINGLES EVER,I love these pringles. they are so good. I alw...
996,Healthy and tasty,This flaxseed mix has a nutty but smooth taste...
997,"Works, economical alternative to other 'diet' ...","This stuff works. As reported by real doctors,..."
998,Great Nylabone,This is the best Nylabone I have bought so far...


In [113]:
df_s["cleaned_text"]=df_s["Text"].apply(lambda x:cleanText(x))

In [114]:
df_s.head()

,Summary,Text,cleaned_text
0,"My wife loves these, so I love them too.",I have ensured my wife's love for me by buying...,ensured wife's love buying one box every pay c...
1,EXCELLENT ANTI -OXIDANT,tHIS POWERFUL FRUIT HAS 2 TIMES THE ANTI-OXIDA...,powerful fruit 2 times anti-oxidant blueberrie...
2,tone's chili powder 20oz shaker,Never had a problem with this brand. i use alo...,never problem brand. use alot chile powder mak...
3,"Better than Peanut M&M""s",I picked these up recently and the are amazing...,picked recently amazing. like sweet stuff afte...
4,What's in this stuff?,What's in this stuff? Where's the list of ing...,what has / what is stuff? where has / where is...


### Performing extractive text summarisation 

#### Using word frequency (TF - IDF)

In [115]:
import nltk
nltk.download('punkt')
df_s['sents'] = df_s['Text'].apply(lambda x: sent_tokenize(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [116]:
df_s['documents_size'] = df_s['sents'].apply(lambda x: len(x)) 

In [117]:
df_s['sents'][1]

['tHIS POWERFUL FRUIT HAS 2 TIMES THE ANTI-OXIDANT OF BLUEBERRIES.',
 'SUN-MAID MALE QUALITY PRODUCTS AND THESE PRUNES ARE SO PLUMP AND JUICY A DELICIOUS TREAT BY THEM SELVES OR USED WITH OTHER FAVORITE FOODS.',
 'YES THESE ARE GOOD IF YOUR CONSTIPATED BUT THEY DO SO MUCH MORE.',
 'TYPE IN BENEFITS OF PRUNES IN YOUR SEARCH ENGINE AND BE AMAZED.',
 'DR. OZ ALSO LIKES THEM.']

In [118]:
# Finding word frequency

def create_frequency_matrix(sentences):
    frequency_matrix = {}

    for sent in sentences:
        freq_table = {}
        words=sent.split()
        for word in words:

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:10]] = freq_table

    return frequency_matrix

In [119]:
df_s['freq_matrix'] = df_s['sents'].apply(lambda x: create_frequency_matrix(x))
#freq_matrix = create_frequency_matrix(sents)

In [120]:
# Finding term frequency

def create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [121]:
df_s['tf_matrix'] = df_s['freq_matrix'].apply(lambda x: create_tf_matrix(x))
#tf_matrix = create_tf_matrix(freq_matrix)

In [122]:
# Finding reviews per words

def create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [123]:
df_s['count_doc_per_words'] = df_s['freq_matrix'].apply(lambda x: create_documents_per_words(x))
#count_doc_per_words = create_documents_per_words(freq_matrix)

In [124]:
# Creating inverse document frequency matrix

def create_idf_matrix(freq_matrix, count_doc_per_words, documents_size):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(documents_size / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [125]:
df_s['idf_matrix'] = df_s.apply(lambda x: create_idf_matrix(x.freq_matrix, x.count_doc_per_words, x.documents_size), axis=1)
#idf_matrix = create_idf_matrix(freq_matrix, count_doc_per_words, documents_size)

In [126]:
# term frequency and inverse document frequency matrix

def create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [127]:
df_s['tf_idf_matrix'] = df_s.apply(lambda x: create_tf_idf_matrix(x.tf_matrix, x.idf_matrix), axis=1)
#tf_idf_matrix = create_tf_idf_matrix(tf_matrix, idf_matrix)

In [128]:
# Calculate sentence scores

def sentence_scores(tf_idf_matrix) -> dict:
    

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score
        if count_words_in_sentence !=0:
            sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence
        else:
            sentenceValue[sent]=0
    return sentenceValue

In [129]:
df_s['sentence_scores'] = df_s['tf_idf_matrix'].apply(lambda x: sentence_scores(x))
#sentence_scores = sentence_scores(tf_idf_matrix)

In [130]:
# finding average to set threshold

def find_average_score(sentenceValue) -> int:
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [131]:
# average sentence score is set as threshold, ..can try other

df_s['threshold'] = df_s['sentence_scores'].apply(lambda x: find_average_score(x))
#find_average_score(sentence_scores)

In [132]:
# Finally Generate Summary

def generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = []

    for sentence in sentences:
        if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] >= (threshold):
            summary.append(sentence)
            sentence_count += 1

    return summary

In [133]:
df_s['gen_summary'] = df_s.apply(lambda x: '.'.join(generate_summary(x.sents, x.sentence_scores,0.8*x.threshold )), axis=1)
#summary = '.'.join(generate_summary(sents, sentence_scores,0.8*threshold ))

In [134]:
# Evaluating the extracted summaries using BLEU score

df_s['b_score'] = df_s.apply(lambda x: sentence_bleu(x.Summary, x.gen_summary), axis = 1)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [135]:
# Calculating the final performance

df_s['b_score_r'] = df_s['b_score'].apply(lambda x: 1 if x >= 0.5 else 0)

In [136]:
d = df_s[['Text', 'Summary', 'gen_summary', 'b_score']]

In [137]:
pd.set_option('display.max_colwidth', None)

In [138]:
d.head(1)

,Text,Summary,gen_summary,b_score
0,I have ensured my wife's love for me by buying her one box of these with every pay check. She finds them the best candy in the galaxy.,"My wife loves these, so I love them too.",She finds them the best candy in the galaxy.,0.707107


In [208]:
d_report = d.iloc[[100, 33], [0, 2]]

In [209]:
d_report.head(3)

,Text,gen_summary
100,"The Reviewers saying: Oh, it's so smooth. No bitterness Here! And what do I do? Buy a pack of 4 only to find out it is not really all that smooth, capisce? Yes, of course I can add water till it feels smooth but I thought this coffee was genuinely smooth. It is not. Besides that it is in the same category of Dunkin Donuts, not more not less and that is not saying much. I am getting to the point that the reviews sell me faster than the product itself and thanks a lot, the majority of times the reviews are right on the money. People that buy knows and I listen. Not in this case! This is not the promised smooth coffee, it is bitter, really. But of course it is being consumed for spoiled we are not. Cheap Bastards is what we are that we are expecting high quality smoothness at such a convenient and inexpensive price. You get what you pay for unless you get extremely lucky and find at bargain prices a superior product. This is not it. By we being CB I am talking about my Tribe, of course and this one gets a solid 3 Stars just like Dunkin Donuts!","The Reviewers saying: Oh, it's so smooth..No bitterness Here!.And what do I do?.It is not..People that buy knows and I listen..Not in this case!.This is not the promised smooth coffee, it is bitter, really..This is not it."
33,"The product tastes great. It tastes like higher end bottled water. One reviewer called the taste ""silky"" and I can't think of a better description for it. It goes down smooth, that's for sure.<br /><br />I'm unsure about certain aspects of this water. On the one hand, I wish I would have had some during the times my youngest suffered from acid reflux or when anyone in the household suffered from an upset stomach. On the other, I'm concerned that drinking too much of it will cause an imbalance with stomach acids, causing further digestion problems.<br /><br />I do think this is an excellent product overall for those who need it. For me, I'm probably going to keep a bottle or two on hand for upset stomachs. I don't think I will be drinking it daily or replacing my current water-of-choice with it.",The product tastes great..It tastes like higher end bottled water.


#### Using cosine similarity

In [211]:
df_s['sents'].iloc[100]

["The Reviewers saying: Oh, it's so smooth.",
 'No bitterness Here!',
 'And what do I do?',
 'Buy a pack of 4 only to find out it is not really all that smooth, capisce?',
 'Yes, of course I can add water till it feels smooth but I thought this coffee was genuinely smooth.',
 'It is not.',
 'Besides that it is in the same category of Dunkin Donuts, not more not less and that is not saying much.',
 'I am getting to the point that the reviews sell me faster than the product itself and thanks a lot, the majority of times the reviews are right on the money.',
 'People that buy knows and I listen.',
 'Not in this case!',
 'This is not the promised smooth coffee, it is bitter, really.',
 'But of course it is being consumed for spoiled we are not.',
 'Cheap Bastards is what we are that we are expecting high quality smoothness at such a convenient and inexpensive price.',
 'You get what you pay for unless you get extremely lucky and find at bargain prices a superior product.',
 'This is not it

In [212]:
#lets create similarity matrix
sim_mat = np.zeros([len(df_s['sents'][100]), len(df_s['sents'][100])])

In [213]:
sim_mat.shape

(16, 16)

In [214]:
"""
creating a similarty matrix
"""
for i in range(len(df_s['sents'][100])):
    for j in range(len(df_s['sents'][100])):
        v1,v2 = embed([df_s['sents'][100][i],df_s['sents'][100][j]])
        v1 = v1.numpy()
        v2 = v2.numpy()
        sim_mat[i][j] = 1 - scipy.spatial.distance.cosine(v1, v2)
"""
After this we count the similarity scores of each sentences with all
other sentences and sort them according to the scores. Idea behind doing this is 
Our top scored sentence is similar to most of the sentences that means out of all the
sentences this sentence has the info which belong to many other sentences also.
"""

'\nAfter this we count the similarity scores of each sentences with all\nother sentences and sort them according to the scores. Idea behind doing this is \nOur top scored sentence is similar to most of the sentences that means out of all the\nsentences this sentence has the info which belong to many other sentences also.\n'

In [215]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(df_s['sents'][100])), reverse=True)
for i in range(3):
    print(ranked_sentences[i][1])

This is not the promised smooth coffee, it is bitter, really.
Buy a pack of 4 only to find out it is not really all that smooth, capisce?
It is not.


In [168]:
"""
Lets put all of them into one function
"""
def get_top_x(l, sentences):
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            v1,v2 = embed([sentences[i],sentences[j]])
            v1 = v1.numpy()
            v2 = v2.numpy()
            sim_mat[i][j] = 1 - scipy.spatial.distance.cosine(v1, v2)
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph, max_iter = 100000000)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    s = ""
    if l < len(ranked_sentences):
        for i in range(l):
            s = s + " " + ranked_sentences[i][3]
        return s
    else:
        return(ranked_sentences)

In [147]:
df_s['gen_summary_tr'] = df_s['sents'].apply(lambda x: get_top_x(1 , x))

KeyboardInterrupt: ignored

In [216]:
data = [['This is not the promised smooth coffee, it is bitter, really. Buy a pack of 4 only to find out it is not really all that smooth, capisce? It is not.'], ['For me, I\'m probably going to keep a bottle or two on hand for upset stomachs. On the other, I\'m concerned that drinking too much of it will cause an imbalance with stomach acids, causing further digestion problems.<br /><br />I do think this is an excellent product overall for those who need it. I don\'t think I will be drinking it daily or replacing my current water-of-choice with it.']]
 

In [227]:
p =  pd.DataFrame(d_report['Text'], columns = ['Text'])

In [236]:
p = p.set_index([pd.Index([0,1])])

In [237]:
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['gen_summary'])

In [238]:
df

,gen_summary
0,"This is not the promised smooth coffee, it is bitter, really. Buy a pack of 4 only to find out it is not really all that smooth, capisce? It is not."
1,"For me, I'm probably going to keep a bottle or two on hand for upset stomachs. On the other, I'm concerned that drinking too much of it will cause an imbalance with stomach acids, causing further digestion problems.<br /><br />I do think this is an excellent product overall for those who need it. I don't think I will be drinking it daily or replacing my current water-of-choice with it."


In [239]:
frames = [p, df]

result = pd.concat(frames, axis = 1)

In [240]:
result

,Text,gen_summary
0,"The Reviewers saying: Oh, it's so smooth. No bitterness Here! And what do I do? Buy a pack of 4 only to find out it is not really all that smooth, capisce? Yes, of course I can add water till it feels smooth but I thought this coffee was genuinely smooth. It is not. Besides that it is in the same category of Dunkin Donuts, not more not less and that is not saying much. I am getting to the point that the reviews sell me faster than the product itself and thanks a lot, the majority of times the reviews are right on the money. People that buy knows and I listen. Not in this case! This is not the promised smooth coffee, it is bitter, really. But of course it is being consumed for spoiled we are not. Cheap Bastards is what we are that we are expecting high quality smoothness at such a convenient and inexpensive price. You get what you pay for unless you get extremely lucky and find at bargain prices a superior product. This is not it. By we being CB I am talking about my Tribe, of course and this one gets a solid 3 Stars just like Dunkin Donuts!","This is not the promised smooth coffee, it is bitter, really. Buy a pack of 4 only to find out it is not really all that smooth, capisce? It is not."
1,"The product tastes great. It tastes like higher end bottled water. One reviewer called the taste ""silky"" and I can't think of a better description for it. It goes down smooth, that's for sure.<br /><br />I'm unsure about certain aspects of this water. On the one hand, I wish I would have had some during the times my youngest suffered from acid reflux or when anyone in the household suffered from an upset stomach. On the other, I'm concerned that drinking too much of it will cause an imbalance with stomach acids, causing further digestion problems.<br /><br />I do think this is an excellent product overall for those who need it. For me, I'm probably going to keep a bottle or two on hand for upset stomachs. I don't think I will be drinking it daily or replacing my current water-of-choice with it.","For me, I'm probably going to keep a bottle or two on hand for upset stomachs. On the other, I'm concerned that drinking too much of it will cause an imbalance with stomach acids, causing further digestion problems.<br /><br />I do think this is an excellent product overall for those who need it. I don't think I will be drinking it daily or replacing my current water-of-choice with it."
